**Proceso de Transformación para la Homologacion de Universidades**

*Tendremos en cuenta los criterios de analisis mencionados*

In [1]:
#Importamos las librerias que se usaran para este previo analisis
import numpy as np
import pandas as pd

#Almacenamos en variables los origenes de nuestros dataset

#Enlace del dataset instituciones_educativas.csv
url_csv = "https://raw.githubusercontent.com/nickiunac/homologar_universidades/main/instituciones_educativas.csv"

#Enlace del dataset Universidades.json
url_json = "https://raw.githubusercontent.com/nickiunac/homologar_universidades/main/Universidades.json"

#Leemos los archivos en los formatos correspondientes

#Dataframe de Instituciones Educativas
df_csv = pd.read_csv(url_csv)

#Dataframe de Universidades
df_json = pd.read_json(url_json)

*No validaremos si los dataset no presentan vacios, ni la cantidad total de registros. Revisar el Script "Analisis y Resultados"*

**Vamos a realizar en 2 partes el analisis**

*parte 1: Analizaremos las siglas coincidentes.*

*parte 2: Analizaremos los registros restantes.*

**PARTE 1**

In [2]:
#Se ha identificado que hay registros que estan separados por "-" sus siglas y algunos son solamente siglas
df_csv[df_csv["value"].str.contains("-")]

,candidateId,value
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb
14,630665b8594e18004af04816,Universidad Privada Del Norte - Upn
25,630665b8594e18004af04864,Universidad Peruana De Ciencias Aplicadas - Upc
...,...,...
8558,634b7b93bf94c20009ce6b21,Instituto De Formación Bancaria - Ifb
8564,633f3b127e47370040358137,Universidad Cesar Vallejo - Ucv
8584,6312a4a8e5ee99003f0863ac,Universidad Cesar Vallejo - Ucv
8585,6312a4a8e5ee99003f0863ac,Universidad Cesar Vallejo - Ucv


*Mencionado lo anterior, hacemos las transformaciones necesarias*

In [3]:
#Elaboramos el Analisis Parte 1

#Creamos un nuevo campo donde aplicamos lambda para cada fila, lo cual dividimos el valor de la columna por el caracter "-"
#si en caso no presenta "-" nos devuelve el valor original de la columna.
df_csv['campo3'] = df_csv.apply(lambda row: row['value'].split('-')[1] if '-' in row['value'] else row['value'], axis=1)

#Eliminamos todos los espacios en blanco
df_csv['campo3'] = df_csv['campo3'].str.replace(r'\s+', '', regex=True)

#Transformamos el nuevo campo en minuscula
df_csv["campo3"] = df_csv["campo3"].str.lower()

#Creamos un nuevo campo de siglas para no alterar el original y transformamos en minuscula
df_json["Siglas2"] = df_json["Siglas "].str.lower()

#Combinamos los dos dataframe con el campo en comun  y usamos un left para mantener todas las filas de df_csv
#lo guardamos en un nuevo dataframe
df_parte1 = pd.merge(df_csv,df_json, left_on="campo3", right_on="Siglas2", how="left")

#Creamos un nuevo campo y Utilizamos la funcion np.where de numpy para darle una condicion donde encuentra NAN ponga "Nombre"
#y hay valor ponga "Sigla"
df_parte1['tipo'] = np.where(df_parte1['Nombre '].isna(), 'Nombre', 'Sigla')

#Eliminamos las columnas que no requerimos
drop_columns=["campo3","código INEI", "Siglas ", "Siglas2"]
df_parte1.drop(columns=drop_columns, inplace = True)

#Mostramos una previsualizacion
df_parte1.head()

,candidateId,value,Nombre,tipo
0,630633fa1e599d0009d6dee0,Ucv,Universidad César Vallejo S.A.C.,Sigla
1,630633fa1e599d0009d6dee0,José Gálvez,NaN,Nombre
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.,Sigla
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,NaN,Nombre
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,NaN,Nombre


In [4]:
#Mostramos la cantidad de registros encontrados
df_parte1[(df_parte1["tipo"] == "Sigla")].shape[0]

590

*Ahora que ya analizamos registros que tienen siglas iguales a las universidades licenciadas, vamos analizar los que son de tipo: "Nombre"*

**Parte 2**

In [5]:
#Elaboramos el Analisis Parte 2

#Creamos un nuevo dataframe y seleccionamos solo los datos "Nombre" del campo "tipo"
df_parte2 = df_parte1[(df_parte1["tipo"] == "Nombre")]

#Creamos una nueva columna para no alterar el valor original y transformamos a minuscula
df_parte2["value2"] = df_parte2["value"].str.lower()

#Nos quedamos con las filas que no contengan las palabras ("instituto","colegio","universidad")
#instituto - colegio: quitamos estos registros ya que la evaluacion es con respecto a universidades
#universidad - quitamos estos registrso, para quedarnos con los que no llevan esa palabra clave e identifcar errores
#ortograficos o la aproximidad que tiene con las universidades licenciadas
df_parte2 = df_parte2[~df_parte2["value2"].str.contains("instituto")]
df_parte2 = df_parte2[~df_parte2["value2"].str.contains("colegio")]
df_parte2 = df_parte2[~df_parte2["value2"].str.contains("universidad")]

#Agregamos la palabra "Universidad" al comienzo de los valores del campo "value"
#Realizamos dicha accion que nos ayudara tener mayor similitud al momento de aplicar las coincidencias y tener mayor similitud
df_parte2["value"] = df_parte2["value"].apply(lambda x: "Universidad "+x)

#Combinamos las columnas de nuestros dos dataframe, pero utilizando el indice como valor clave y esto es debido a que
#existe duplicados en el campo candidateId, es por eso que manejamos atraves del indice.
df_parte2 = pd.merge(df_parte1,df_parte2, left_index=True, right_index=True, how="left")

#Mostramos la previsualizacion
df_parte2.head()

C:\Users\Ricardo\AppData\Local\Temp\ipykernel_1980\3007746444.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parte2["value2"] = df_parte2["value"].str.lower()


,candidateId_x,value_x,Nombre _x,tipo_x,candidateId_y,value_y,Nombre _y,tipo_y,value2
0,630633fa1e599d0009d6dee0,Ucv,Universidad César Vallejo S.A.C.,Sigla,NaN,NaN,NaN,NaN,NaN
1,630633fa1e599d0009d6dee0,José Gálvez,NaN,Nombre,630633fa1e599d0009d6dee0,Universidad José Gálvez,NaN,Nombre,josé gálvez
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.,Sigla,NaN,NaN,NaN,NaN,NaN
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,NaN,Nombre,63063bda29da5b00089fb224,Universidad Centro Peruano De Estudios Bancar...,NaN,Nombre,centro peruano de estudios bancarios - cepeban
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,NaN,Nombre,NaN,NaN,NaN,NaN,NaN


In [6]:
#Eliminamos las columnas que no requerimos
drop_columns=["candidateId_y","Nombre _y", "tipo_y", "value2"]
df_parte2.drop(columns=drop_columns, inplace = True)

#Mostramos la previsualizacion
df_parte2.head()

,candidateId_x,value_x,Nombre _x,tipo_x,value_y
0,630633fa1e599d0009d6dee0,Ucv,Universidad César Vallejo S.A.C.,Sigla,NaN
1,630633fa1e599d0009d6dee0,José Gálvez,NaN,Nombre,Universidad José Gálvez
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.,Sigla,NaN
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,NaN,Nombre,Universidad Centro Peruano De Estudios Bancar...
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,NaN,Nombre,NaN


In [7]:
#Definimos la función lambda para evaluar los valores de value_y y value_x al encontrar un registro nulo
def evaluar_campos(row):
    if pd.isnull(row['value_y']):
        return row['value_x']
    else:
        return row['value_y']

#Aplicamos la funcion a cada fila del DataFrame
df_parte2['value_y'] = df_parte2.apply(lambda row: evaluar_campos(row), axis=1)

#Mostramos la previsualizacion
df_parte2.head()

,candidateId_x,value_x,Nombre _x,tipo_x,value_y
0,630633fa1e599d0009d6dee0,Ucv,Universidad César Vallejo S.A.C.,Sigla,Ucv
1,630633fa1e599d0009d6dee0,José Gálvez,NaN,Nombre,Universidad José Gálvez
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.,Sigla,Universidad Peruana De Ciencias Aplicadas - Upc
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,NaN,Nombre,Universidad Centro Peruano De Estudios Bancar...
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,NaN,Nombre,Instituto De Formación Bancaria - Ifb


*Aqui ya unimos nuestros dos analisis, contamos los registros para validar sino falta un registro*

In [8]:
#Contamos los registros totales
df_parte2.shape[0]

8588

*Coincide con nuestro df_csv inicial de 8588 registros*

*Ahora realizaremos una transformacion con una herramienta util para la comparacion de cadenas de texto de manera difusa o aproximada que es la biblioteca de Python **fuzzywuzzy**, esta herramienta nos ayudara a calcular la similitud entre dos cadenas de texto.*

*Como hicimos en el script "Analisis y Resultados"*

In [9]:
#Transformamos en minuscula los campos que se van a comparar
df_parte2["value_y"] = df_parte2["value_y"].str.lower()
df_json["Nombre "] = df_json["Nombre "].str.lower()

In [10]:
#Instalamos la biblioteca fuzzywuzzy

In [11]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [12]:
#Importamos las funciones
from fuzzywuzzy import fuzz, process

#Funcion para homologar universidades (para encontrar la mejor coincidencia)
#En este caso usamos el algoritmo "token_sort_radio" y realizamos una condicion
#donde se tiene que obtener una similitud mayor o igual a 90 y devuelve el valor, sino devuelve vacio
def homologar_universidad(value_y):
    match = process.extractOne(value_y, df_json['Nombre '], scorer=fuzz.token_sort_ratio)
    if match[1] >= 90:
        return match[0]
    else:
        return ''

#Creamos una columna de universidad homologada en df_csv para saber si pertenece a una universidad licenciada
df_parte2['Universidad_Homologada'] = df_parte2['value_y'].apply(homologar_universidad)

#Mostramos la previsualizacion
df_parte2.head()

C:\Users\Ricardo\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,candidateId_x,value_x,Nombre _x,tipo_x,value_y,Universidad_Homologada
0,630633fa1e599d0009d6dee0,Ucv,Universidad César Vallejo S.A.C.,Sigla,ucv,
1,630633fa1e599d0009d6dee0,José Gálvez,NaN,Nombre,universidad josé gálvez,
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.,Sigla,universidad peruana de ciencias aplicadas - upc,
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,NaN,Nombre,universidad centro peruano de estudios bancar...,
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,NaN,Nombre,instituto de formación bancaria - ifb,


In [13]:
#Definimos la función lambda para evaluar los valores de Universidad_Homologada y Nombre _x al encontrar un registro nulo
def evaluar_campos(row):
    if row['Universidad_Homologada'] == "":
        return row['Nombre _x']
    else:
        return row['Universidad_Homologada']

#Aplicamos la funcion a cada fila del DataFrame
df_parte2['Universidad_Homologada'] = df_parte2.apply(lambda row: evaluar_campos(row), axis=1)

#Mostramos la previsualizacion
df_parte2.head()

,candidateId_x,value_x,Nombre _x,tipo_x,value_y,Universidad_Homologada
0,630633fa1e599d0009d6dee0,Ucv,Universidad César Vallejo S.A.C.,Sigla,ucv,Universidad César Vallejo S.A.C.
1,630633fa1e599d0009d6dee0,José Gálvez,NaN,Nombre,universidad josé gálvez,NaN
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.,Sigla,universidad peruana de ciencias aplicadas - upc,Universidad Peruana de Ciencias Aplicadas S.A.C.
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,NaN,Nombre,universidad centro peruano de estudios bancar...,NaN
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,NaN,Nombre,instituto de formación bancaria - ifb,NaN


In [14]:
#Eliminamos las columnas que no requerimos
drop_columns=["Nombre _x","value_y"]
df_parte2.drop(columns=drop_columns, inplace = True)

#Mostramos la previsualizacion
df_parte2.head()

,candidateId_x,value_x,tipo_x,Universidad_Homologada
0,630633fa1e599d0009d6dee0,Ucv,Sigla,Universidad César Vallejo S.A.C.
1,630633fa1e599d0009d6dee0,José Gálvez,Nombre,NaN
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Sigla,Universidad Peruana de Ciencias Aplicadas S.A.C.
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,Nombre,NaN
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,Nombre,NaN


In [15]:
#Llenamos los valores nulos por la palabra "No Homologada"
df_parte2["Universidad_Homologada"] = df_parte2["Universidad_Homologada"].fillna("No Homologada")

#Mostramos la previsualizacion
df_parte2.head()

,candidateId_x,value_x,tipo_x,Universidad_Homologada
0,630633fa1e599d0009d6dee0,Ucv,Sigla,Universidad César Vallejo S.A.C.
1,630633fa1e599d0009d6dee0,José Gálvez,Nombre,No Homologada
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Sigla,Universidad Peruana de Ciencias Aplicadas S.A.C.
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,Nombre,No Homologada
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,Nombre,No Homologada


*Ahora contamos cuantos registros encontro*

In [16]:
#Contamos cuantos registros coincidentes obtuvimos
df_parte2[(df_parte2["Universidad_Homologada"] != "No Homologada")].shape[0]

2202

**Entonces aplicando la herramienta fuzzywuzzy y transformando la data de acuerdo al analisis mencionado, nos ayudo a obtener 2202 registros**

*****************************

**CON ESTA INFORMACION Y LA MAS OPTIMA, ELABORAREMOS LOS SCRIPT SOLICITADOS POR EL RETO**

**1. Generar archivo: universidades_homologadas.csv**

In [17]:
#Sacamos una copia para no alterar el original
df_script_1 = df_parte2.copy()

#Eliminamos las columnas que no requerimos
drop_columns=["tipo_x"]
df_script_1.drop(columns=drop_columns, inplace = True)

#Renombramos las columnas de acuerdo a lo solicitado
df_script_1 = df_script_1.rename(columns={'candidateId_x': 'candidateId', 'value_x': 'value', 'Universidad_Homologada': 'universidad homologada'})

#Mostramos la previsualizacion
df_script_1.head()

,candidateId,value,universidad homologada
0,630633fa1e599d0009d6dee0,Ucv,Universidad César Vallejo S.A.C.
1,630633fa1e599d0009d6dee0,José Gálvez,No Homologada
2,63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.
3,63063bda29da5b00089fb224,Centro Peruano De Estudios Bancarios - Cepeban,No Homologada
4,630644da1e599d0009d6e2f4,Instituto De Formación Bancaria - Ifb,No Homologada


In [18]:
#Digita tu ruta para guardar el archivo en formato csv
ruta = "G:/Krowdy/Homologación de Universidades/03. OUTPUT/"

df_script_1.to_csv(ruta + 'universidades_homologadas.csv', index=False, encoding=None)

***************************

**2. Generar archivo: sinonimo_universidades.json**

In [19]:
# Agrupamos por cluster y aplicamos la función set a la columna "value_x" para eliminar duplicados
df_script_2 = df_parte2.groupby('Universidad_Homologada')['value_x'].agg(set).reset_index()

#Convertir "value_x" a listas y asignamos la lista resultante
df_script_2['value_x'] = df_script_2['value_x'].apply(list)

#Renombramos las columnas de acuerdo a lo solicitado
df_script_2.columns = ['nombre_universidad', 'sinonimos']

#Quitamos el registro "No Homologada" del cambio nombre_universidad y reseteamos el indice
df_script_2 = df_script_2[df_script_2["nombre_universidad"] != "No Homologada"].reset_index(drop=True)

#Mostramos la previsualizacion
df_script_2.head()

,nombre_universidad,sinonimos
0,Asociación Civil Universidad de Ciencias y Hum...,"[Uch, Universidad De Ciencias Y Humanidades - ..."
1,Pontificia Universidad Católica del Perú,[Pucp]
2,Universidad Andina del Cusco,[Uac]
3,Universidad Autónoma del Perú S.A.C.,[Universidad Autónoma Del Perú - Ua]
4,Universidad Católica Santo Toribio de Mogrovejo,[Usat]


*Para realizar la estructurada solicitada, debemos almacenarlo como un diccionario y luego recien pasar al formato JSON*

In [20]:
#Creamos una lista vacía para almacenar los diccionarios
universidades = []

#Iteramos por cada fila del DataFrame
for index, row in df_script_2.iterrows():
    
    # Creamos un diccionario para cada universidad
    universidad = {}
    universidad['nombre_universidad'] = row['nombre_universidad']
    universidad['sinonimos'] = row['sinonimos']
    
    # Agregamos el diccionario a la lista
    universidades.append(universidad)

#Convertimos la lista de diccionarios a formato JSON y guardar en un archivo

import json

#Digita tu ruta para guardar el archivo en formato json segun la estructura solicitada en el reto
ruta = "G:/Krowdy/Homologación de Universidades/03. OUTPUT/"

with open(ruta + 'sinonimo_universidades.json', 'w') as file:
    json.dump(universidades, file)